# Check MRI Quality

Things that need to be adjusted for SEA Lab:
* File paths (of course)
* Modify to account for multiple runs of a sequence (need to know what that looks like scanner side)


In [20]:
# Participant-specific information
subject_id = '999'

mri_location = '/Users/catcamacho/Box/Vandy-Stanford MR comparison/Vanderbilt/'
raw_data_locations = {'rest0': mri_location + subject_id + '/NIFTI/*rsEPI-PE0_MB6R1_*.PAR', 
                      'rest1': mri_location + subject_id + '/NIFTI/*rsEPI-PE1_MB6R1_*.PAR', 
                      'dwi0': mri_location + subject_id + '/NIFTI/*DWI_MB3R1_pe0*', 
                      'dwi1': mri_location + subject_id + '/NIFTI/*DWI_MB3R1_pe1*',
                      'dwi': mri_location + subject_id + '/NIFTI/*DTI_b700_2000_96dir*',
                      'qt1': mri_location + subject_id + '/NIFTI/*qT1*', 
                      't1w': mri_location + subject_id + '/NIFTI/*AnatBrain_T13D*',
                      't2w': mri_location + subject_id + '/NIFTI/*3D_Brain_VIEW_T2*', 
                      'pcasl_pw': mri_location + subject_id + '/NIFTI/*pCASL_PLD1900_LD1000_REL5FIXED.01_real*', 
                      'pcasl_m0': mri_location + subject_id + '/NIFTI/*pCASL_REL5FIXED_M0.01*'}


### Reorganize and rename data to conform to BIDS

In [22]:
# Import libraries and set up static study variables
import os
from glob import glob
from subprocess import check_call

sequence_list = list(raw_data_locations.keys())

# make level 1 folders
os.mkdir(mri_location + subject_id + '/anat')
os.mkdir(mri_location + subject_id + '/func')
os.mkdir(mri_location + subject_id + '/dwi')

for sequence in sequence_list:
    if 'dwi' in sequence:
        dwis0 = glob(raw_data_locations['dwi0'])
        for file in dwis0:
            filepath, filebasename = os.path.split(file)
            filename, fileext = os.path.splitext(filebasename)
            os.rename(file, mri_location + subject_id + '/dwi/'+subject_id+'_dwi_pe0'+fileext)
        dwis1 = glob(raw_data_locations['dwi1'])
        for file in dwis1:
            filepath, filebasename = os.path.split(file)
            filename, fileext = os.path.splitext(filebasename)
            os.rename(file, mri_location + subject_id + '/dwi/'+subject_id+'_dwi_pe1'+fileext)
        dwis = glob(raw_data_locations['dwi'])
        for file in dwis:
            filepath, filebasename = os.path.split(file)
            filename, fileext = os.path.splitext(filebasename)
            os.rename(file, mri_location + subject_id + '/dwi/'+subject_id+'_dwi_pe0_b2000_b700'+fileext)
    elif 'rest' in sequence:
        
    elif 'pcasl' in sequence:
        pws = glob(raw_data_locations['pcasl_pw'])
        for file in pws:
            filepath, filebasename = os.path.split(file)
            filename, fileext = os.path.splitext(filebasename)
            os.rename(file, mri_location + subject_id + '/func/'+subject_id+'_pcasl_pw'+fileext)
        m0s = glob(raw_data_locations['pcasl_m0'])
        for file in m0s:
            filepath, filebasename = os.path.split(file)
            filename, fileext = os.path.splitext(filebasename)
            os.rename(file, mri_location + subject_id + '/func/'+subject_id+'_pcasl_m0'+fileext)
    elif sequence=='t1w':
        t1ws = glob(raw_data_locations['t1w'])
        for file in t1ws:
            filepath, filebasename = os.path.split(file)
            filename, fileext = os.path.splitext(filebasename)
            os.rename(file, mri_location + subject_id + '/anat/'+subject_id+'_T1w'+fileext)
    else sequence=='t2w':
        t2ws = glob(raw_data_locations['t2w'])
        for file in t2ws:
            filepath, filebasename = os.path.split(file)
            filename, fileext = os.path.splitext(filebasename)
            os.rename(file, mri_location + subject_id + '/anat/'+subject_id+'_T2w'+fileext)
        
# zip up all the niftis to take up less space
niftis = glob(mri_location + subject_id + '/*/*.nii')
for nii in niftis:
    check_call(['gzip',nii]) 
    
# Tar up the extra sequences to take up less space
os.rename(mri_location + subject_id +'/NIFTI', mri_location + subject_id +'/misc')
os.rename(mri_location + subject_id +'/XMLPARREC', mri_location + subject_id +'/raw')
check_call(['tar','-zcvf',mri_location + subject_id +'/misc.tgz',mri_location + subject_id +'/misc'])
check_call(['tar','-zcvf',mri_location + subject_id +'/raw.tgz',mri_location + subject_id +'/raw'])

## Produce QC Report
This section does the following:
* Create motion summaries for rest and DTI
* Print slice montages for each main sequence
* create histograms for ASL and qT1 values